# 25 ml

uprava imdb datasetu, aby sa mohol vpouzit na vytvorenie dodatocneho kontexu

In [16]:
from datetime import datetime
import pandas as pd
import json

In [17]:
# depth of this file in the project
file_depth = '../../..'
imdb_dataset = '/dataset/imdb/imdb_original/'

In [18]:
# basics = pd.read_csv(file_depth + imdb_dataset + 'title.basics.tsv',sep='\t')

# print('unique tipes: ', basics['titleType'].unique())
# print('count: ', basics.shape[0])
# # remove non movie like rows
# basic_movies = basics[ (basics['titleType'] == 'movie') | (basics['titleType'] == 'tvMovie') | (basics['titleType'] == 'tvMiniSeries') | (basics['titleType'] == 'tvSpecial')]
# print('unique tipes: ', basic_movies['titleType'].unique())
# print('count: ', basic_movies.shape[0])

-----

In [19]:
with open(file_depth + '/config/data_25m_config.json') as config_file:
    config = json.load(config_file)

# dataset = config['dataset_path']

In [20]:
original_csv_movies_data = config['original_csv_movies_data']
movies = pd.read_csv(file_depth + original_csv_movies_data, encoding="UTF-8")
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [21]:
original_csv_ratings_data = config['original_csv_ratings_data']
ratings = pd.read_csv(file_depth + original_csv_ratings_data, encoding="UTF-8")
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


#### Context from rating timestamp

In [22]:
context = pd.DataFrame()
context['rating_id'] = ratings.index
context['timestamp'] = ratings['timestamp']
context

,rating_id,timestamp
0,0,1147880044
1,1,1147868817
2,2,1147868828
3,3,1147878820
4,4,1147868510
...,...,...
25000090,25000090,1240953372
25000091,25000091,1240951998
25000092,25000092,1240950697
25000093,25000093,1240953434


In [23]:
# create datetime object from timestamp
context['date'] = context['timestamp'].apply(lambda x: datetime.fromtimestamp(x))

In [24]:
# create a week day column from timestamp

"""
0: Monday
1: Tuesday
2: Wednesday
3: Thursday
4: Friday
5: Saturday
6: Sunday
"""
context['day'] = context['timestamp'].apply(lambda x: datetime.fromtimestamp(x).isoweekday())

In [25]:
# create a is_weekday column from day column

"""
0: false / weekend
1: true  / weekday
"""
def get_is_weekday(day):
    if day == 6 or day == 7:
        return 0
    else:
        return 1
    
context['is_weekday'] = context['day'].apply(get_is_weekday)

In [26]:
"""
1: Spring	
2: Summer
3: Fall
4: Winter
"""
def get_season(month):
    if month >= 3 and month <= 5:
        return 1
    elif month >= 6 and month <= 8:
        return 2
    elif month >= 9 and month <= 11:
        return 3
    else:
        return 4
    
context['season'] = context['date'].apply(lambda x: get_season(x.month))

In [27]:
"""
1 - Morning
2 - Afternoon
3 - Evening
4 - Night
"""
def get_part_of_day(hour):
    if hour >= 5 and hour < 12:
        return 1
    elif hour >= 12 and hour < 17:
        return 2
    elif hour >= 17 and hour < 21:
        return 3
    else:
        return 4
    
context['part_of_day'] = context['date'].apply(lambda x: get_part_of_day(x.hour))

In [30]:
with open('holidays.json', 'r') as json_file:
    holiday_dates = json.load(json_file)

def is_date_in_interval(date, start_date, end_date, holiday_name):
    if holiday_name == 'new_years': 
        if date.month == 12:
            if date.day >= start_date.day:
                return True

        elif date.month == 1:
            if date.day <= end_date.day:
                return True
        return False
    else:
        if date.month >= start_date.month and date.month <= end_date.month:
            if date.day >= start_date.day and date.day <= end_date.day:
                return True
        return False


def find_holiday(date):
    for holiday_name, interval in holiday_dates.items():
        start_date = datetime.strptime(interval['start'], '%m-%d')
        end_date = datetime.strptime(interval['end'], '%m-%d')
        if is_date_in_interval(date, start_date, end_date, holiday_name):
            return holiday_name
    return 'no_holiday'

In [31]:
# new_context_path = config['new_context_path']
# context.to_csv(file_depth + new_context_path +  'context.csv', index=False)

In [32]:
context['holiday'] = context['date'].apply(find_holiday)

In [33]:
holiday_counts = context['holiday'].value_counts()
holiday_counts

holiday
no_holiday        18972894
summer_holiday     4247812
christmas           731008
thanksgiving        534591
new_years           245197
valentines          188197
veterans_day         80396
Name: count, dtype: int64

In [34]:
context = context.drop(columns=['date'])
context = context.drop(columns=['timestamp'])

In [35]:
context

,rating_id,day,is_weekday,season,part_of_day,holiday
0,0,3,1,1,3,no_holiday
1,1,3,1,1,2,no_holiday
2,2,3,1,1,2,no_holiday
3,3,3,1,1,3,no_holiday
4,4,3,1,1,2,no_holiday
...,...,...,...,...,...,...
25000090,25000090,2,1,1,4,no_holiday
25000091,25000091,2,1,1,4,no_holiday
25000092,25000092,2,1,1,4,no_holiday
25000093,25000093,2,1,1,4,no_holiday


In [36]:
new_context_path = config['new_context_path']
context.to_csv(file_depth + new_context_path +  'context.csv', index=False)

#### Movies

In [15]:
movies['movie_year'] = movies['title'].str.extract(r'\((\d{4})\)')

print('Does any movie have no year informatrion? : ' + str(movies['movie_year'].isnull().values.any()))
print('Unique years: ')
print(movies['movie_year'].unique())

Does any movie have no year informatrion? : True
Unique years: 
['1995' '1994' '1996' '1976' '1992' '1988' '1967' '1993' '1964' '1977'
 '1965' '1982' '1990' '1991' '1989' '1937' '1940' '1969' '1981' '1973'
 '1970' '1960' '1955' '1959' '1968' '1980' '1975' '1986' '1948' '1943'
 '1950' '1946' '1987' '1997' '1974' '1956' '1958' '1949' '1972' '1998'
 '1933' '1952' '1951' '1957' '1961' '1954' '1934' '1944' '1963' '1942'
 '1941' '1953' '1939' '1947' '1945' '1938' '1935' '1936' '1926' '1932'
 '1985' '1979' '1971' '1978' '1966' '1962' '1983' '1984' '1931' '1922'
 '1999' '1927' '1929' '1930' '1928' '1925' '1914' '2000' '1919' '1923'
 '1920' '1918' '1921' '2001' '1924' '2002' '2003' '1915' '2004' '1916'
 '1917' '2005' '2006' '1902' '1903' '2007' '2008' '2009' '1912' '2010' nan
 '1913' '2011' '1898' '1899' '1894' '2012' '1910' '2013' '1896' '2014'
 '2015' '1895' '1909' '1911' '1900' '2016' '2017' '2018' '2019' '1905'
 '1904' '1891' '1892' '1908' '1897' '1887' '1888' '1890' '1878' '1874'
 '1901' '

There are 410 movies with no year value

In [14]:
movies[movies['movie_year'].isnull()]

,movieId,title,genres,movie_year
15036,79607,"Millions Game, The (Das Millionenspiel)",Action|Drama|Sci-Fi|Thriller,NaN
25387,123619,Terrible Joe Moran,(no genres listed),NaN
26284,125571,The Court-Martial of Jackie Robinson,(no genres listed),NaN
26309,125632,In Our Garden,(no genres listed),NaN
26392,125958,Stephen Fry In America - New World,(no genres listed),NaN
...,...,...,...,...
62071,207714,Tales of Found Footage,(no genres listed),NaN
62104,207884,Enduring Destiny,(no genres listed),NaN
62285,208597,Punk the Capital: Building a Sound Movement,Documentary,NaN
62326,208763,Yosemite: The Fate of Heaven,(no genres listed),NaN


There are also 5000 movies with no genre

In [ ]:
# create datetime object from timestamp
context['date'] = context['timestamp'].apply(lambda x: datetime.fromtimestamp(x))

In [8]:
movies['genres'].unique()

array(['Adventure|Animation|Children|Comedy|Fantasy',
       'Adventure|Children|Fantasy', 'Comedy|Romance', ...,
       'Children|Comedy|Fantasy|Mystery', 'Children|Comedy|Crime|Fantasy',
       'Comedy|Horror|Mystery|Sci-Fi|Western'], dtype=object)

In [ ]:
# Change genre Children's to ForChildren
movies['genres'] = movies['genres'].str.replace("Children's", "ForChildren")

# Extract year from movie title and return it as context
movies['movie_year'] = movies['title'].str.extract(r'\((\d{4})\)')

print('Does any movie have no year informatrion? : ' + str(movies['movie_year'].isnull().values.any()))
print('Unique years: ')
print(movies['movie_year'].unique())